In [ ]:
using JSON, Distributions, ProgressMeter, Distances, Plots, DataFrames, Clustering, LightGraphs, SimpleWeightedGraphs, Colors, GraphPlot

Here's where you want to input your mon vs mon matrix. You'll probably also need to change the get_name function below

In [ ]:
vs_matrix = 

In [ ]:
function get_distance_matrix(vs_matrix)
    d_matrix = pairwise(Euclidean(), vs_matrix) 
    d_matrix = convert(Array{T} where T <: AbstractFloat, d_matrix)
    return d_matrix
end

In [ ]:
d_matrix = get_distance_matrix(vs_matrix)

In [ ]:
means = []
stds = []
@showprogress for n = 2:length(meta)
    scores = []
    for j = 1:10_000
        clusters = kmeans(vs_matrix, n)
        a = assignments(clusters)
        c = counts(clusters)
        score = mean(silhouettes(a, c, d_matrix))
        push!(scores, score)
    end
    push!(means, mean(scores))
    push!(stds, std(scores))
end

In [ ]:
n = 10

In [ ]:
function get_submatrix(group1, group2)
    n1 = length(group1)
    n2 = length(group2)
    submatrix = zeros(n1, n2)
    for i in 1:n1, j in 1:n2
        submatrix[i, j] = vs_matrix[group1[i], group2[j]]
    end
    return submatrix
end

In [ ]:
function generate_graph(clusters; arrow_standard = 0.65)
    n = nclusters(clusters)
    a = assignments(clusters)
    g = SimpleDiGraph(n)
    scoreWeights = []
    winWeights = []
    for i = 1:(n - 1)
        for j = (i + 1):n
            i_team = findall(x -> x == i, a)
            j_team = findall(x -> x == j, a)
            submatrix = get_submatrix(i_team, j_team)
            avgScore = mean(submatrix)
            winPercentage = sum(submatrix .> 0.5) / length(submatrix)
            if avgScore > arrow_standard && winPercentage > arrow_standard
                add_edge!(g, i, j)
                push!(scoreWeights, 2 * (avgScore - 0.5))
                push!(winWeights, 2 * (winPercentage - 0.5))
            elseif avgScore < 1 - arrow_standard && winPercentage < 1 - arrow_standard
                add_edge!(g, j, i)
                push!(scoreWeights, 2 * (0.5 - avgScore))
                push!(winWeights, 2 * (0.5 - winPercentage))
            end
        end
    end
    return g, scoreWeights, winWeights
end

In [ ]:
get_name(i) = meta[i]

In [ ]:
score = 0
while true
    clusters = kmeans(vs_matrix, n)
    a = assignments(clusters)
    c = counts(clusters)
    score = mean(silhouettes(a, c, d_matrix))
    if score > means[n] + 2 * stds[n]
        println(score)
        for i = 1:n
            group = findall(x -> x == i, a)
            print("Group " * string(i))
            println(get_name.(group))
        end
        global c = c
        global g, scoreWeights, winWeights = generate_graph(clusters, arrow_standard = 0.65)
        break
    end
end

In [ ]:
gplot(g, nodelabel = 1:length(c), nodesize = c, edgelinewidth = winWeights, edgestrokec = weighted_color_mean.(scoreWeights, Ref(colorant"black"), Ref(colorant"white")))